In [1]:
import sys

sys.path.insert(0, 'compiled_protobufs')

In [2]:
""" Generate Taskgraphs from datasets """

from models_datasets.recipe_1m_model import AbstractModelDataset
from models_datasets.recipe_1m_model import Recipe1MModel
from models_datasets.wikihow_model import WikihowModel

dataset_models = {
    "COOKING" : Recipe1MModel,
    "DIY": WikihowModel,
}

for model_name, model in dataset_models.items():
    dataset_model: AbstractModelDataset = model()
    print(f"Dataset Model: {model_name}")
    dataset_model.convert_to_taskgraphs()

Dataset Model: COOKING
Taskgraphs already converted. Call overwrite=True in convert_to_taskgraphs() to overwrite the preprocessed taskgraphs.
Dataset Model: DIY
Taskgraphs already converted. Call overwrite=True in convert_to_taskgraphs() to overwrite the preprocessed taskgraphs.


In [3]:
# pd_cooking_queries = pd.read_csv("queries/cooking.csv")
# ids = [f'query-{i}' for i in range(0, len(pd_cooking_queries))]
# pd_cooking_queries.insert(0, 'id', ids)
# pd_cooking_queries.head()
# pd_cooking_queries.to_csv("queries/cooking2.csv",index=False)


In [4]:
""" Build indexes from datasets """

# from models_indexes.bm25_model import BM25Model

# bm25_food = BM25Model(domain = "COOKING", t5=True)
# bm25_food.build_index()

from models_indexes.ance_model import AnceModel
ance_diy = AnceModel(domain = "DIY")

# bm25_cooking = BM25Model(domain = "DIY", t5=True)
# bm25_cooking.build_index()

In [6]:
""" Load queries """
import pandas as pd

import os
# queries_path = os.path.join(os.getcwd(), "queries", "cooking.csv")

# pd_cooking_queries = pd.read_csv(queries_path).iloc[:10]
# bm25_food.convert_search_results_to_run(pd_cooking_queries)
# bm25_food.create_empty_judgments(pd_cooking_queries, 5)
queries_path = os.path.join(os.getcwd(), "queries", "diy.csv")
pd_diy_queries = pd.read_csv(queries_path).iloc[:10]
pd_diy_queries

,raw query,target query,domain,knowledge,reasoning
0,how to choose a good hirstyle,choose good hairstyle,diy,NaN,NaN
1,how to learn a new language,learn new language,diy,NaN,NaN
2,how to clean carpet,clean carpet,diy,NaN,NaN
3,how to wash a car,wash car,diy,NaN,NaN
4,how do I decorate an office,decorate office,diy,NaN,NaN
5,help me paint a wall,paint a wall,diy,NaN,NaN
6,how do you remove crown mold from a wall,remove crown molding wall,diy,NaN,NaN
7,how to make homemade toothpaste,homemade toothpaste,diy,NaN,NaN
8,how do I clean the fridge,clean fridge,diy,NaN,NaN
9,how can I screw in a nail,screw nail,diy,NaN,NaN
